In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
%matplotlib inline

/home/kim/leadership-practice-practice/lib/python3.7/site-packages/pandas/compat/__init__.py:85: UserWarning: Could not import the lzma module. Your installed Python is incomplete. Attempting to use lzma compression will result in a RuntimeError.
  warnings.warn(msg)
/home/kim/leadership-practice-practice/lib/python3.7/site-packages/pandas/compat/__init__.py:85: UserWarning: Could not import the lzma module. Your installed Python is incomplete. Attempting to use lzma compression will result in a RuntimeError.
  warnings.warn(msg)


In [7]:
posts = pd.read_csv('../data/raw_data/posts.csv')
users = pd.read_csv('../data/raw_data/users.csv')

In [8]:
users.sample(5)

,uid,first_name,last_name,sex,is_leader
7642,1770555,Мария ...,Рожниковская,1,0
24839,255200769,Дмитрий ...,Рекрут,2,0
13828,4527423,Алина ...,Басова,1,0
33272,3709472,Данил ...,Асфандияров,2,0
15046,1885594,Полина ...,Петрова,1,0


In [9]:
posts.sample(5)

,from_id,owner_id,text,comments,likes,reposts
3266546,68310465,68310465,"Скоро новый год, ёлка,запах мандаринов и подар...",0,14,0
561369,83895043,83895043,😘😘😘😘 М О С К О В С К И Й ...,0,6,0
1875428,233195399,233195399,Кикимора повержена! Отметь мой успех и получи ...,0,3,0
2099666,500909,500909,Поддержим!,0,11,1
1912779,162807927,162807927,Новость от Валерия Анашвили:\nГРЭМ ХАРМАН. Имм...,0,4,0


In [10]:
valid_users = np.unique(users['uid'])

In [11]:
posts = posts[posts['from_id'].isin(valid_users)]
posts['text'] = posts['text'].astype('str')
data = posts.groupby(['from_id'], sort=True).agg({'text': ' '.join,
                                                  'comments': 'sum',
                                                  'likes': 'sum',
                                                  'reposts': 'sum'})

In [12]:
users.index = users.uid
users.drop('uid', axis=1, inplace=True)

In [13]:
data['is_leader'] = users.loc[data.index]['is_leader']

In [14]:
X = data.drop('is_leader', axis=1)
Y = data['is_leader'].values

In [16]:
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.linear_model import LogisticRegression

tfidf = TfidfVectorizer()
model = LogisticRegression()


model.fit(tfidf.fit_transform(X['text']), Y)

/home/kim/leadership-practice-practice/lib/python3.7/site-packages/sklearn/linear_model/logistic.py:432: FutureWarning: Default solver will be changed to 'lbfgs' in 0.22. Specify a solver to silence this warning.
  FutureWarning)


LogisticRegression(C=1.0, class_weight=None, dual=False, fit_intercept=True,
                   intercept_scaling=1, l1_ratio=None, max_iter=100,
                   multi_class='warn', n_jobs=None, penalty='l2',
                   random_state=None, solver='warn', tol=0.0001, verbose=0,
                   warm_start=False)

In [17]:
preds = model.predict(tfidf.transform(X['text']))

In [18]:
submission = pd.DataFrame({'is_leader':preds}, index=X.index)

submission.to_csv('../data/submission/train_submission.csv')